## Finding Averages and Ranges for Global Mortality Data
#### Camilo Gomez and Daniel Verdear for Safe-esteem

Mortality data has been provided for several global regions. The relevant splits on this data are cause of death, victim age, and victim gender. The contents of this data file must be parsed to obtain an understanding of the data and how it will affect scores in the Safe-esteem app.

In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dfs = pd.read_excel('Selected_Data_Sample_01_2019.xlsx', sheet_name = list(range(1,12)))

In [3]:
dfs[8]

,measure_id,measure_name,location_id,location_name,sex_id,sex_name,age_id,age_name,cause_id,cause_name,metric_id,metric_name,year,val,upper,lower
0,1,Deaths,63,Ukraine,1,Male,5,1 to 4,344,Neglected tropical diseases and malaria,3,Rate,2016,0.021246,0.041032,0.010341
1,1,Deaths,63,Ukraine,2,Female,5,1 to 4,344,Neglected tropical diseases and malaria,3,Rate,2016,0.014702,0.032993,0.006501
2,1,Deaths,63,Ukraine,1,Male,6,5 to 9,344,Neglected tropical diseases and malaria,3,Rate,2016,0.016010,0.025922,0.010024
3,1,Deaths,63,Ukraine,2,Female,6,5 to 9,344,Neglected tropical diseases and malaria,3,Rate,2016,0.012911,0.020287,0.008165
4,1,Deaths,63,Ukraine,1,Male,7,10 to 14,344,Neglected tropical diseases and malaria,3,Rate,2016,0.013493,0.018455,0.009741
5,1,Deaths,63,Ukraine,2,Female,7,10 to 14,344,Neglected tropical diseases and malaria,3,Rate,2016,0.013822,0.019434,0.010049
6,1,Deaths,63,Ukraine,1,Male,8,15 to 19,344,Neglected tropical diseases and malaria,3,Rate,2016,0.014789,0.021129,0.009767
7,1,Deaths,63,Ukraine,2,Female,8,15 to 19,344,Neglected tropical diseases and malaria,3,Rate,2016,0.012284,0.017238,0.008541
8,1,Deaths,63,Ukraine,1,Male,9,20 to 24,344,Neglected tropical diseases and malaria,3,Rate,2016,0.027920,0.045699,0.016884
9,1,Deaths,63,Ukraine,2,Female,9,20 to 24,344,Neglected tropical diseases and malaria,3,Rate,2016,0.012758,0.018544,0.008180


Line 2 does a really cool thing. It imports the entire Excel workbook, sheet by sheet, into a data structure called an OrderedDict. This preserves the order of the sheets, keeps them separate, and makes them iterable. This data structure will make it easy to search through every region with several split conditions.
<b>NOTE:</b> The operation on Line 2 is extremely resource consuming. It must run through thousands of lines of data and build multiple data structures to contain them all. It took several minutes on a machine with a 4GHz processor and 16GB RAM.

The following cell contains the definitions for functions to query the data based on age, gender, and cause of death. They were written by Camilo and are the most essential component of this analysis.

In [4]:
#rewritten to take the dataframe index as a parameter and search through dfs[index][sex id, age id, cause id]

def risk_search(index, sex_id, age_id, domain):
    '''takes in demographic category and returns avergage risk score

    '''
        
    result_df = dfs[index].loc[(dfs[index]['sex_id'] == sex_id) & (dfs[index]['age_id'] == age_id) & 
                               (dfs[index]['cause_id'] == domain)]
    return result_df
    
def average_rate(result_df):
    '''returns average of val for any constructed dataframe'''
    return result_df['val'].mean()
def range_int(result_df):
    ''' returns range that the values of a particular group may take
    '''
    range_int = (result_df['upper'].max(), result_df['lower'].min())
    return range_int

In [5]:
testing_res = risk_search(1,1,9,696)
#should return a dataframe for all men ages 20-24 in the USA that dies of 'Unintentional injuries'
print(testing_res.head())
print(testing_res.shape)

      measure_id measure_name  location_id location_name  sex_id sex_name  \
2425           1       Deaths          546     Minnesota       1     Male   
3001           1       Deaths          570    Washington       1     Male   
4640           1       Deaths          548      Missouri       1     Male   
5696           1       Deaths          573       Wyoming       1     Male   
6814           1       Deaths          538          Iowa       1     Male   

      age_id  age_name  cause_id              cause_name  metric_id  \
2425       9  20 to 24       696  Unintentional injuries          3   
3001       9  20 to 24       696  Unintentional injuries          3   
4640       9  20 to 24       696  Unintentional injuries          3   
5696       9  20 to 24       696  Unintentional injuries          3   
6814       9  20 to 24       696  Unintentional injuries          3   

     metric_name  year        val      upper      lower  
2425        Rate  2015   7.743803   9.449444   6.244

We now need to explain why there are 102 instances when instances should be organized by state. By looking through the data, we can see that Washington DC is counted separately and data from 2015 and 2016 are stored. 

(50+1)*2 = 102 so everything adds up, and our functions work!

In [6]:
usa_healthd = []

In [7]:
for j in [295,409,718]:
    for k in [1,2]:
        for l in list(range(5,21)):
            dfw = risk_search(1,k,l,j)
            dfw.reset_index(inplace=True)
            
            if(dfw.empty):
                continue
            loc = 'USA'
            sex = dfw['sex_name'].iloc[0]
            age = dfw['age_name'].iloc[0]
            cause = dfw['cause_name'].iloc[0]
            avg = average_rate(dfw)
            rng = range_int(dfw)
            usa_healthd.append([loc,sex,age,cause,avg,rng])
            
usa_health = pd.DataFrame(usa_healthd, columns = ['location','sex','age','cause','average','range'])

In [8]:
world_healthd = []

In [9]:
for i in list(range(2,12)):
    for j in [295,409,718]:
        for k in [1,2]:
            for l in list(range(5,21)):
                dfw = risk_search(i,k,l,j)
                dfw.reset_index(inplace=True)

                if(dfw.empty):
                    continue
                loc = dfw['location_name'].iloc[0]
                if(i==2):
                    loc = 'Mexico'
                if(i==4):
                    loc = 'Brazil'
                sex = dfw['sex_name'].iloc[0]
                age = dfw.loc[0,'age_name']
                cause = dfw.loc[0,'cause_name']
                avg = average_rate(dfw)
                rng = range_int(dfw)
                world_healthd.append([loc,sex,age,cause,avg,rng])
            
world_health = pd.DataFrame(world_healthd, columns = ['location','sex','age','cause','average','range'])

In [10]:
usa_health.head()

,location,sex,age,cause,average,range
0,USA,Male,1 to 4,"Communicable, maternal, neonatal, and nutritio...",2.984085,"(5.67998522994, 1.27501435121999)"
1,USA,Male,5 to 9,"Communicable, maternal, neonatal, and nutritio...",0.788709,"(1.77734434934999, 0.252776363889)"
2,USA,Male,10 to 14,"Communicable, maternal, neonatal, and nutritio...",0.687146,"(1.3910870627, 0.219197884201)"
3,USA,Male,15 to 19,"Communicable, maternal, neonatal, and nutritio...",0.970994,"(2.37937673641, 0.436667580826999)"
4,USA,Male,20 to 24,"Communicable, maternal, neonatal, and nutritio...",2.052460,"(5.5182089963, 0.95430637052)"


In [11]:
world_health.head()

,location,sex,age,cause,average,range
0,Mexico,Male,1 to 4,"Communicable, maternal, neonatal, and nutritio...",18.747181,"(79.9124968738, 6.27351055638)"
1,Mexico,Male,5 to 9,"Communicable, maternal, neonatal, and nutritio...",4.287553,"(14.5889266152999, 1.45238541666)"
2,Mexico,Male,10 to 14,"Communicable, maternal, neonatal, and nutritio...",3.705016,"(9.48495503051, 1.74779269610999)"
3,Mexico,Male,15 to 19,"Communicable, maternal, neonatal, and nutritio...",4.467666,"(10.9634196754, 2.53017773039999)"
4,Mexico,Male,20 to 24,"Communicable, maternal, neonatal, and nutritio...",9.740475,"(19.879040332, 5.28076288746)"


This is the basic structure for creating these dataframes.

Copy paste cells 6-9 to compute Crime, Stationary Accidents, Walking Accidents, Cycling Accidents, Motor Vehicle Accidents. Just rewrite the list in the j for loop to include the criteria that are required.

Additionally, you can use pointwise multiplication to calculate the values for Cycling and Motor Vehicle Crime. Hope this helps!!